In [1]:
from binance.spot import Spot
import pandas as pd
import requests
import math

api_key = "WVDAJGRZSOaXmeh94MA1b1pqYLbkvudD0iHTgGPNxu4PYEKQD4SHcFAt3ZQPdFHG"
api_secret = "o6DnBjwXkY0psTVHbxI8ipZZdeaW4dIWuEqbukbs7FFPQ9z8aps9BOsopiNNsHP1"


client = Spot(api_key=api_key, api_secret=api_secret)

def get_balance_account():

    def get_asset_price(asset):
        url = f'https://api.binance.com/api/v3/ticker/price?symbol={asset}USDT'
        response = requests.get(url)
        data = response.json()
        return float(data.get('price', 0))

    data = client.account()
    balances = data['balances']
    rows = []
    for balance in balances:
        free_balance = round(float(balance['free']), 4)
        if free_balance > 0:
            asset = balance['asset']
            try:
                price = get_asset_price(asset.replace("LD","")) 
                value = round(free_balance * price, 2)  
            except Exception as e:
                print(f"Error fetching price for {asset}: {e}")
                price = 0
                value = 0
            row = {
                'Mã Token': asset,
                'Số lượng token': free_balance,
                'Giá trị $': value  
            }
            rows.append(row)
    balances_df = pd.DataFrame(rows)
    return balances_df



In [2]:
print(get_balance_account())

ClientError: (401, -2015, 'Invalid API-key, IP, or permissions for action.', {'Content-Type': 'application/json;charset=UTF-8', 'Content-Length': '70', 'Connection': 'keep-alive', 'Date': 'Wed, 31 Jul 2024 13:46:56 GMT', 'Server': 'nginx', 'x-mbx-uuid': '99e494c2-35a8-461a-b145-c0c93cae1cd8', 'x-mbx-used-weight': '342', 'x-mbx-used-weight-1m': '342', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains', 'X-Frame-Options': 'SAMEORIGIN', 'X-Xss-Protection': '1; mode=block', 'X-Content-Type-Options': 'nosniff', 'Content-Security-Policy': "default-src 'self'", 'X-Content-Security-Policy': "default-src 'self'", 'X-WebKit-CSP': "default-src 'self'", 'Cache-Control': 'no-cache, no-store, must-revalidate', 'Pragma': 'no-cache', 'Expires': '0', 'X-Cache': 'Error from cloudfront', 'Via': '1.1 1d7fb40804b60d5a9f4cfe1063c9fa1a.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'HKG54-P1', 'X-Amz-Cf-Id': 'R7KWe8Llu2mq0-GNsczS4uY0hpnW-kYs9ayBkP-63GMFAnqyUoFI9Q=='}, None)

In [ ]:
def buy_order(symbol,volume):

    def get_asset_price(asset):
        url = f'https://api.binance.com/api/v3/ticker/price?symbol={asset}'
        response = requests.get(url)
        data = response.json()
        return float(data.get('price', 0))
    
    volume = str(round(float(volume)/get_asset_price(symbol),4))

    params = {
        'symbol': symbol,
        'side': 'BUY',  
        'type': 'MARKET',
        'quantity': volume
    }
    
    response = client.new_order(**params)
    return response
print(buy_order("MATICUSDT","100"))

ClientError: (400, -1013, 'Filter failure: LOT_SIZE', {'Content-Type': 'application/json;charset=UTF-8', 'Content-Length': '47', 'Connection': 'keep-alive', 'Date': 'Mon, 29 Jul 2024 12:37:30 GMT', 'Expires': '0', 'Server': 'nginx', 'x-mbx-uuid': '1c30df41-2079-4951-9348-4decda181303', 'x-mbx-used-weight': '65', 'x-mbx-used-weight-1m': '65', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains', 'X-Frame-Options': 'SAMEORIGIN', 'X-Xss-Protection': '1; mode=block', 'X-Content-Type-Options': 'nosniff', 'Content-Security-Policy': "default-src 'self'", 'X-Content-Security-Policy': "default-src 'self'", 'X-WebKit-CSP': "default-src 'self'", 'Cache-Control': 'no-cache, no-store, must-revalidate', 'Pragma': 'no-cache', 'X-Cache': 'Error from cloudfront', 'Via': '1.1 38bc5b2316e0e5caebb0bd430db7b956.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'HAN50-P2', 'X-Amz-Cf-Id': 'SpNixDz0G-wRhpd9tHxNUuVx9UjC3sl-tt7Dv0VigUQkFlAoSQevQQ=='}, None)

In [ ]:
print(get_balance_account())

   Mã Token  Số lượng token  Giá trị $
0      USDT        760.9591       0.00
1     LDBNB          0.0024       1.42
2     LDETH          0.0001       0.34
3       SOL          0.5210     100.01
4     LDFXS         25.0899      58.56
5     LDLDO         36.5791       0.00
6      LDOP         33.9688      59.24
7     LDRAD         45.9196      59.37
8    LDLUNC          0.0015       0.00
9    LDRARE        701.2284      61.29
10    LDRIF        626.4942      72.11
11  LDPOLYX          1.7586       0.49
12   LDBICO        334.6331     105.41
13      LDT       4610.1916     114.52
14   LDHIFI         90.5962      47.36
15    LDSUI         75.1999      58.30
16  LDPIXEL          0.0312       0.01
17   LDSTRK        109.0741      59.88
18   LDBOME       6791.4200      72.14
19      LDW        868.7914     250.30
20   BANANA          0.0001       0.01


In [ ]:

def sell_order(symbol):
    df = get_balance_account()
    quantity = df[df['Mã Token'] == symbol.replace("USDT","")]['Số lượng token'].values[0]
    quantity = quantity[:-3]
    for i in range(1,5):
        volume = volume[:-i]
        try:
            params = {
                'symbol': symbol,
                'side': 'SELL',  
                'type': 'MARKET',
                "quantity" : quantity
            }
            
            response = client.new_order(**params)
            print(f"\n Đặt lệnh bán thành công {symbol} với khối lượng {volume} coin.")
            break
        except : pass
print(sell_order("MATICUSDT"))

0.518
{'symbol': 'SOLUSDT', 'orderId': 6892429677, 'orderListId': -1, 'clientOrderId': 'hs7OopS2PEhrWsLH4TN0cP', 'transactTime': 1722256513948, 'price': '0.00000000', 'origQty': '0.51800000', 'executedQty': '0.51800000', 'cummulativeQuoteQty': '99.35240000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'SELL', 'workingTime': 1722256513948, 'fills': [{'price': '191.80000000', 'qty': '0.51800000', 'commission': '0.09935240', 'commissionAsset': 'USDT', 'tradeId': 618342481}], 'selfTradePreventionMode': 'EXPIRE_MAKER'}
